# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_data = pd.read_csv (r'../output_data/city_data.csv')
print (city_data)

              City  Longitude  Latitude  Max Temperature  Humidity  \
0        Nantucket   -70.0995   41.2835            73.96        90   
1          Fortuna  -124.1573   40.5982            71.85        72   
2          Rikitea  -134.9692  -23.1203            72.64        83   
3          Kushima   131.2333   31.4583            80.64        89   
4        Carnarvon   113.6333  -24.8667            64.47        63   
..             ...        ...       ...              ...       ...   
533  Florianópolis   -48.5012  -27.6146            65.25        88   
534          Gweta    25.2333  -20.1833            67.50        17   
535          Lagoa   -27.9833   39.0500            71.42        83   
536          Erzin    36.2000   36.9559            81.32        79   
537         Qrendi    14.4583   35.8347            82.11        94   

     Cloudiness  Wind Speed Country        Date  
0            90       10.36      US  1628547712  
1             1       16.11      US  1628547712  
2        

In [4]:
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Longitude,Latitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Nantucket,-70.0995,41.2835,73.96,90,90,10.36,US,1628547712
1,Fortuna,-124.1573,40.5982,71.85,72,1,16.11,US,1628547712
2,Rikitea,-134.9692,-23.1203,72.64,83,55,8.81,PF,1628547712
3,Kushima,131.2333,31.4583,80.64,89,43,4.70,JP,1628547713
4,Carnarvon,113.6333,-24.8667,64.47,63,90,13.80,AU,1628547713
...,...,...,...,...,...,...,...,...,...
533,Florianópolis,-48.5012,-27.6146,65.25,88,0,12.66,BR,1628547952
534,Gweta,25.2333,-20.1833,67.50,17,2,2.68,BW,1628547952
535,Lagoa,-27.9833,39.0500,71.42,83,4,7.54,PT,1628547953
536,Erzin,36.2000,36.9559,81.32,79,0,0.85,TR,1628547953


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [22]:
# Create a list of coordinates
coordinates = city_data_df[['Latitude','Longitude']]

# Create a list of city humidities
humidity = city_data_df['Humidity'].astype(float)

In [72]:
# Customize the size of the figure
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='900px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [42]:
# Determine ideal weather conditions

# Max Temperature: 65-85 F
# Wind Speed: Less than 10 mph
# Cloudiness: Less than 10%
# Humidity: Less than 50%

ideal_weather_df = city_data_df.loc[(city_data_df["Max Temperature"] >= 65) & (city_data_df["Max Temperature"] <= 85) &
                                    (city_data_df["Wind Speed"] <= 10) & 
                                    (city_data_df["Cloudiness"] <= 10) &
                                    (city_data_df["Humidity"] <= 50)].dropna()

ideal_weather_df

,City,Longitude,Latitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
94,Gandajika,23.9500,-6.7500,77.77,28,1,3.29,CD,1628547756
148,Viking,-111.7684,53.1001,80.46,22,2,7.67,CA,1628547775
171,Korla,86.1469,41.7597,74.98,42,2,8.01,CN,1628547786
177,Nicolás Bravo,-88.9167,18.4500,83.28,45,9,8.55,MX,1628547790
192,Kaniama,24.1833,-7.5667,73.58,31,0,4.05,CD,1628547800
204,Arroyo de la Encomienda,-4.7969,41.6096,74.14,36,0,6.91,ES,1628547805
225,Erbil,44.0106,36.1926,82.22,23,0,1.14,IQ,1628547815
289,Abhā,42.5053,18.2164,75.02,25,0,0.00,SA,1628547843
368,Kaka,59.6143,37.3482,80.91,15,0,3.13,TM,1628547875
413,Alta Floresta,-56.0861,-9.8756,84.16,29,8,2.10,BR,1628547895


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [50]:
hotel_df = ideal_weather_df.copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Longitude,Latitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
94,Gandajika,23.9500,-6.7500,77.77,28,1,3.29,CD,1628547756,
148,Viking,-111.7684,53.1001,80.46,22,2,7.67,CA,1628547775,
171,Korla,86.1469,41.7597,74.98,42,2,8.01,CN,1628547786,
177,Nicolás Bravo,-88.9167,18.4500,83.28,45,9,8.55,MX,1628547790,
192,Kaniama,24.1833,-7.5667,73.58,31,0,4.05,CD,1628547800,
204,Arroyo de la Encomienda,-4.7969,41.6096,74.14,36,0,6.91,ES,1628547805,
225,Erbil,44.0106,36.1926,82.22,23,0,1.14,IQ,1628547815,
289,Abhā,42.5053,18.2164,75.02,25,0,0.00,SA,1628547843,
368,Kaka,59.6143,37.3482,80.91,15,0,3.13,TM,1628547875,
413,Alta Floresta,-56.0861,-9.8756,84.16,29,8,2.10,BR,1628547895,


In [61]:
# Locate API
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Create params dictionary
params = {"type" : "hotel",
          "keyword": "hotel",
          "radius" : 5000,
          "key" : g_key}

In [62]:
for index, row in hotel_df.iterrows():
    # Get latitude, longitude, and city name from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    # Add location to params dictionary
    params["location"] = f"{lat},{lng}"

    # Create url and make API requests
    print(f"Retrieving results for index {index}: {city_name}.")
    response = requests.get(url, params=params).json()
    
    # Extract results
    results = response['results']
    
    # Save the hotel names to dataframe
    try:
        print(f"Closest hotel to {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # If there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving results for index 94: Gandajika.
Missing field/result... skipping.
------------
Retrieving results for index 148: Viking.
Closest hotel to Viking is Viking Lamplighter Motel.
------------
Retrieving results for index 171: Korla.
Closest hotel to Korla is The Garden Hotel Korla.
------------
Retrieving results for index 177: Nicolás Bravo.
Missing field/result... skipping.
------------
Retrieving results for index 192: Kaniama.
Missing field/result... skipping.
------------
Retrieving results for index 204: Arroyo de la Encomienda.
Closest hotel to Arroyo de la Encomienda is Hotel Vincci Frontaura.
------------
Retrieving results for index 225: Erbil.
Closest hotel to Erbil is Divan Erbil Hotel.
------------
Retrieving results for index 289: Abhā.
Closest hotel to Abhā is Shafa Abha Hotel.
------------
Retrieving results for index 368: Kaka.
Missing field/result... skipping.
------------
Retrieving results for index 413: Alta Floresta.
Closest hotel to Alta Floresta is Hotel 

In [63]:
hotel_df

,City,Longitude,Latitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
94,Gandajika,23.9500,-6.7500,77.77,28,1,3.29,CD,1628547756,Gandajika
148,Viking,-111.7684,53.1001,80.46,22,2,7.67,CA,1628547775,Viking Lamplighter Motel
171,Korla,86.1469,41.7597,74.98,42,2,8.01,CN,1628547786,The Garden Hotel Korla
177,Nicolás Bravo,-88.9167,18.4500,83.28,45,9,8.55,MX,1628547790,Nicolás Bravo
192,Kaniama,24.1833,-7.5667,73.58,31,0,4.05,CD,1628547800,Kaniama
204,Arroyo de la Encomienda,-4.7969,41.6096,74.14,36,0,6.91,ES,1628547805,Hotel Vincci Frontaura
225,Erbil,44.0106,36.1926,82.22,23,0,1.14,IQ,1628547815,Divan Erbil Hotel
289,Abhā,42.5053,18.2164,75.02,25,0,0.00,SA,1628547843,Shafa Abha Hotel
368,Kaka,59.6143,37.3482,80.91,15,0,3.13,TM,1628547875,Kaakhka
413,Alta Floresta,-56.0861,-9.8756,84.16,29,8,2.10,BR,1628547895,Hotel Mandino


In [64]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [73]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='900px'))